<a href="https://colab.research.google.com/github/dosthzyno/DL/blob/main/RNN/RNN_TextGeneration_%EA%B3%BC%EC%A0%9C_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##신입기수 교육: RNN Assignment
: text generation using RNN

: 데이터 이해/전처리를 읽고 난 후 모델 및 Text Generation 함수를 설계해보기

: 참고) text generation using RNN.ipynb파일, https://wikidocs.net/45101


**1) 데이터 이해/전처리**

In [1]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import files
uploaded = files.upload()

Saving ArticlesApril2018.csv to ArticlesApril2018.csv


In [3]:
# ArticlesApril2018.csv 파일 로드
df = pd.read_csv('ArticlesApril2018.csv')

In [5]:
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [4]:
print('Number of columns: ', len(df.columns))
print(df.columns)

Number of columns:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [6]:
df['headline'].isnull().values.any()

False

In [7]:
# 모든 기사의 제목 하나의 리스트로 저장
headline = []
headline.extend(list(df.headline.values))
headline[:10]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [ ]:
# headline 중 unknown 처리

print('총 샘플의 개수 : {}'.format(len(headline))) # 현재 샘플의 개수
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

총 샘플의 개수 : 1324
노이즈값 제거 후 샘플의 개수 : 1214


In [8]:
headline[:10]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [9]:
import re

# 데이터 전처리: 구두점 제거, lowercase()
def data_preprocessing(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    return text

cnt=0
for i in headline:
    headline[cnt] = data_preprocessing(i)
    cnt+=1
headline[:10]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [10]:
# 토큰화 및 단어 집합 크기 확인
t= Tokenizer()
t.fit_on_texts(headline)
vocab_size = len(t.word_index) + 1

print('단어 집합의 크기: {}'.format(vocab_size))

단어 집합의 크기: 3495


In [11]:
# 정수 인코딩 & 하나의 문장을 2 이상 길이의 훈련 데이터로 만들기"
sequences = []
for text in headline:
    for line in text.split('\n'):    # \n 기준으로 문장 토큰화 (한칸 뛰는것 기준)
        encoded = t.texts_to_sequences([line])[0]
        for i in range(1, len(encoded)):
            sequence = encoded[:i+1]   # 길이가 2 이상인 gram들 저장
            sequences.append(sequence)

cnt = 0
for i in sequences:
    if cnt < 11:
        cnt+=1
        print(i)

[100, 270]
[100, 270, 372]
[100, 270, 372, 1116]
[100, 270, 372, 1116, 583]
[100, 270, 372, 1116, 583, 53]
[100, 270, 372, 1116, 583, 53, 7]
[100, 270, 372, 1116, 583, 53, 7, 2]
[100, 270, 372, 1116, 583, 53, 7, 2, 373]
[100, 270, 372, 1116, 583, 53, 7, 2, 373, 11]
[100, 270, 372, 1116, 583, 53, 7, 2, 373, 11, 1117]
[101, 3]


In [12]:
print(sequences)

[[100, 270], [100, 270, 372], [100, 270, 372, 1116], [100, 270, 372, 1116, 583], [100, 270, 372, 1116, 583, 53], [100, 270, 372, 1116, 583, 53, 7], [100, 270, 372, 1116, 583, 53, 7, 2], [100, 270, 372, 1116, 583, 53, 7, 2, 373], [100, 270, 372, 1116, 583, 53, 7, 2, 373, 11], [100, 270, 372, 1116, 583, 53, 7, 2, 373, 11, 1117], [101, 3], [101, 3, 1118], [101, 3, 1118, 2], [101, 3, 1118, 2, 15], [101, 3, 1118, 2, 15, 584], [101, 3, 1118, 2, 15, 584, 25], [101, 3, 1118, 2, 15, 584, 25, 1119], [101, 3, 1118, 2, 15, 584, 25, 1119, 374], [101, 3, 1118, 2, 15, 584, 25, 1119, 374, 375], [101, 3, 1118, 2, 15, 584, 25, 1119, 374, 375, 5], [101, 3, 1118, 2, 15, 584, 25, 1119, 374, 375, 5, 1120], [1, 15], [1, 15, 1121], [1, 15, 1121, 1122], [16, 2], [16, 2, 585], [16, 2, 585, 4], [16, 2, 585, 4, 216], [16, 2, 585, 4, 216, 376], [16, 2, 585, 4, 216, 376, 1123], [16, 2, 585, 4, 216, 376, 1123, 2], [16, 2, 585, 4, 216, 376, 1123, 2, 377], [16, 2, 585, 4, 216, 376, 1123, 2, 377, 586], [9, 168], [9, 16

In [13]:
# 빈도수 상위 단어 확인
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 576번 단어 : {}'.format(index_to_word[576]))
print('빈도수 상위 1번 단어 : {}'.format(index_to_word[1]))

빈도수 상위 576번 단어 : dying
빈도수 상위 1번 단어 : the


In [14]:
# 가장 긴 샘플의 길이에 전체 샘플 길이 맞추기
max_len = max(len(l) for l in sequences) 
print('샘플 최대 길이 : {}'.format(max_len))

# 전체 샘플 길이를 max_len으로 패딩 작업
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

print()
print('패딩 결과 확인')
print(sequences)

샘플 최대 길이 : 24

패딩 결과 확인
[[   0    0    0 ...    0  100  270]
 [   0    0    0 ...  100  270  372]
 [   0    0    0 ...  270  372 1116]
 ...
 [   0    0    0 ...    9 3494  116]
 [   0    0    0 ... 3494  116    2]
 [   0    0    0 ...  116    2 1026]]


In [15]:
# 훈련 데이터에서 레이블 분리 (각 문장의 마지막 단어만 분리해서 y로)
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print('X 확인')
print(X)
print()
print('y 확인')
print(y)

# 레이블에 대한 one-hot encoding
y = to_categorical(y, num_classes = vocab_size)

print()
print('one-hot encoding : y 확인')
print(y)

X 확인
[[   0    0    0 ...    0    0  100]
 [   0    0    0 ...    0  100  270]
 [   0    0    0 ...  100  270  372]
 ...
 [   0    0    0 ...  171    9 3494]
 [   0    0    0 ...    9 3494  116]
 [   0    0    0 ... 3494  116    2]]

y 확인
[ 270  372 1116 ...  116    2 1026]

one-hot encoding : y 확인
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


**2) 모델 설계하기**

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from keras.layers import LSTM

In [24]:
# FILL IN 파트 - RNN 모델 설계 직접해보기 

model = Sequential()

model.add(Embedding(vocab_size, 32, input_length = max_len-1)) # 각 단어를 32차원으로 embedding, 레이블 분리 후 길이는 -1
model.add(SimpleRNN(128))   # 모델 및 은닉 상태 크기 설정 
model.add(Dense(vocab_size, activation = 'softmax')) #Dense Layer 및 활성화 함수 설정
          
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Loss, Optimizer, Metrics 설정 

model.fit(X,y,epochs = 100, verbose=2) #최종 Accuracy가 최소 95% 이상은 나오게 epochs 설정

Epoch 1/100
244/244 - 7s - loss: 7.6434 - accuracy: 0.0267 - 7s/epoch - 31ms/step
Epoch 2/100
244/244 - 4s - loss: 7.0802 - accuracy: 0.0363 - 4s/epoch - 18ms/step
Epoch 3/100
244/244 - 4s - loss: 6.7719 - accuracy: 0.0492 - 4s/epoch - 18ms/step
Epoch 4/100
244/244 - 4s - loss: 6.3955 - accuracy: 0.0582 - 4s/epoch - 18ms/step
Epoch 5/100
244/244 - 4s - loss: 6.0019 - accuracy: 0.0645 - 4s/epoch - 17ms/step
Epoch 6/100
244/244 - 4s - loss: 5.6308 - accuracy: 0.0779 - 4s/epoch - 17ms/step
Epoch 7/100
244/244 - 4s - loss: 5.2764 - accuracy: 0.0936 - 4s/epoch - 17ms/step
Epoch 8/100
244/244 - 4s - loss: 4.9324 - accuracy: 0.1196 - 4s/epoch - 18ms/step
Epoch 9/100
244/244 - 4s - loss: 4.5907 - accuracy: 0.1615 - 4s/epoch - 18ms/step
Epoch 10/100
244/244 - 4s - loss: 4.2608 - accuracy: 0.2133 - 4s/epoch - 18ms/step
Epoch 11/100
244/244 - 4s - loss: 3.9388 - accuracy: 0.2695 - 4s/epoch - 18ms/step
Epoch 12/100
244/244 - 4s - loss: 3.6358 - accuracy: 0.3258 - 4s/epoch - 18ms/step
Epoch 13/100


In [25]:
# 문장 생성 함수
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre') # 데이터에 대한 패딩
        predict_x=model.predict(encoded, verbose=2) 
        classes_x=np.argmax(predict_x,axis=1)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == classes_x: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [26]:
# Text Generation 결과 확인 
# 임의의 단어 'good'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'good', 10))

1/1 - 0s - 209ms/epoch - 209ms/step
1/1 - 0s - 25ms/epoch - 25ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 20ms/epoch - 20ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
good going to in this has a new york hit the
